In [2]:
%pip install yfinance
%pip install talipp
import torch
import numpy
from ensemble import Ensemble
from ensembleTrainer import EnsembleTrainer
import dataReader
import yfinance
import talipp
from trader import Trader
WINDOW_SIZE = 30
HIDDEN_SIZE = 128
LAYER_COUNT = 2
CNN_KERNEL = 6
HYBRID_PROJ = 16
HYBRID_CNN = 8
MLP_DEPTH = 4
STOCK_FILE = '../YFinanceStockData/BTC.csv'
PREDICTION_OFFSET = 1
START_DATE = '2016-1-1'
DATE_RANGE = 365*2
STEPS = 1000

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Using cached talipp-1.9.1-py3-none-any.whl (43 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
input_data, label_data = dataReader.readStockData(STOCK_FILE,WINDOW_SIZE,PREDICTION_OFFSET,START_DATE,DATE_RANGE,binary=False,percent=True,normalize=True)

optimizer = torch.optim.SGD
loss_func = torch.nn.MSELoss()

kwargs = {
    'lr' : 0.00001,
    'momentum': 0.8
}

ensembleModel = Ensemble(WINDOW_SIZE,HIDDEN_SIZE,LAYER_COUNT,CNN_KERNEL,HYBRID_PROJ,HYBRID_CNN,MLP_DEPTH)
ensembleTrainer = EnsembleTrainer(optimizer=optimizer,model=ensembleModel,loss_func=loss_func,**kwargs)
ensembleLosses = ensembleTrainer.train(input_data,label_data,STEPS)

  0%|          | 5/1000 [00:02<08:49,  1.88it/s]


KeyboardInterrupt: 

In [35]:
shortTrader = Trader(True)
longTrader = Trader(False)
buyAndHold = Trader(True)

test_data, test_labels = dataReader.readStockData(STOCK_FILE,WINDOW_SIZE,PREDICTION_OFFSET,'2017-1-1',DATE_RANGE,binary=False,normalize=True,percent=True)
unnorm_data, unnorm_labels = dataReader.readStockData(STOCK_FILE,WINDOW_SIZE,PREDICTION_OFFSET,'2017-1-1',DATE_RANGE,percent=True,normalize=False)
ELEMENT_COUNT = test_data.shape[0] - WINDOW_SIZE - PREDICTION_OFFSET + 1
shortTrader.initialCost = unnorm_data[0][0][0]
longTrader.initialCost = unnorm_data[0][0][0]
buyAndHold.initialCost = unnorm_data[0][0][0]
lastPrice = 0
buyAndHold.buyStock(unnorm_data[0][0][0])
for i in range(ELEMENT_COUNT):
    Prediction = ensembleTrainer.model(test_data[i]).item()
    day_of = unnorm_data[i][0][WINDOW_SIZE - 1].item()
    next_price = Prediction * day_of / 100
    shortTrader.actOnPrediction(day_of,next_price,verbose=False)
    longTrader.actOnPrediction(day_of,next_price,verbose=False)
    lastPrice = day_of

shortTrader.closePositions(lastPrice)
longTrader.closePositions(lastPrice)
buyAndHold.closePositions(lastPrice)
print("Long:")
longTrader.printStats()
print("Short:")
shortTrader.printStats()
print("Const")
buyAndHold.printStats()
print(ensembleLosses[-1])


Long:
Final Net Balance Change: $0.00
Highest Balance: $0.00
Lowest Balance: $0.00
Biggest Single Play: $0.00
Worst Single Play: $0.00
With 0 total transactions
0 Long Positions
0 Short Positions
0.00% Change from initial
Short:
Final Net Balance Change: $-3046.87
Highest Balance: $0.00
Lowest Balance: $-3046.87
Biggest Single Play: $0.00
Worst Single Play: $-3046.87
With 1 total transactions
0 Long Positions
1 Short Positions
-298.20% Change from initial
Const
Final Net Balance Change: $2995.52
Highest Balance: $2995.52
Lowest Balance: $0.00
Biggest Single Play: $2995.52
Worst Single Play: $0.00
With 1 total transactions
1 Long Positions
0 Short Positions
293.18% Change from initial
114.01026153564453
